In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
## 설정
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS     = 40
BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(
    dataset = datasets.FashionMNIST(
        root      = './.data/', 
        train     = True,
        download  = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081))
        ])
    ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = datasets.FashionMNIST(
        root      = './.data/', 
        train     = False,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081))
        ])
    ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [20]:
## 학습
class CNN(nn.Module):
    def  __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.dropout = nn.Dropout2d()
        
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # 커널 크기 2로 지정했습니다.
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = x.view(-1, 320) # x를 1차원으로 변경합니다.
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x) # 0 ~ 9 까지의 출력값을 가진 신경망입니다.
        
        return x

In [21]:
## 파라미터 설정
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) 

In [28]:
## 학습
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
## 평가
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [29]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.579612
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.564555
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.963754
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.774788
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.639304
[1] Test Loss: 0.5106, Accuracy: 80.71%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.711811
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.545363
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.591984
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.507244
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.539360
[2] Test Loss: 0.4836, Accuracy: 81.98%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.609585
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.775569
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.489686
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.588142
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.526070
[3] Test Loss: 0.4735, Accuracy: 82.50%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.519792
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.573853
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.467829


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.314804
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.352859
[29] Test Loss: 0.3313, Accuracy: 88.17%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.300624
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.508126
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.294046
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.559906
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.276163
[30] Test Loss: 0.3292, Accuracy: 87.95%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.358776
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.480497
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.368163
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.383848
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.230797
[31] Test Loss: 0.3259, Accuracy: 88.29%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.282927
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.434057
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.494420
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.364804
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.822783
[32] Test Loss: 0.3224, A